<a href="https://colab.research.google.com/github/datasci-rahul/personal/blob/main/Zero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!nvidia-smi

Sun May 12 14:49:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0              28W /  70W |    221MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
!pip install git+https://github.com/huggingface/diffusers.git
!pip install transformers accelerate torch


  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-u1afj6sf
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-u1afj6sf
  Resolved https://github.com/huggingface/diffusers.git to commit 98ba18ba55f1f74b3d3b51fbe1db2c6978c9f9ed
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [19]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video

pipe = DiffusionPipeline.from_pretrained("cerspense/zeroscope_v2_576w", torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()
pipe.enable_vae_slicing()
pipe.unet.enable_forward_chunking(chunk_size=1, dim=1) # disable if enough memory as this slows down significantly

prompt = "Darth Vader surfing"
video_frames = pipe(prompt, num_inference_steps=40, height=320, width=576, num_frames=36).frames
#video_path = export_to_video(video_frames)

unet/diffusion_pytorch_model.safetensors not found


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

In [23]:
!pip install opencv-python

In [25]:
print(type(video_frames))

<class 'numpy.ndarray'>


In [26]:
video_frames = torch.from_numpy(video_frames)

In [35]:
video_frames = video_frames.to(torch.uint8)

In [30]:
import cv2
import torch
import numpy as np

def export_to_video(video_frames, output_video_path, fps):
    # Convert the tensor to a NumPy array
    video_frames = video_frames.numpy()

    # Convert the NumPy array to the appropriate data type
    video_frames = video_frames.astype(np.uint8)

    # Extract the height and width from the first frame
    h, w, c = video_frames[0][0].shape

    # Create a video writer
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps=fps, frameSize=(w, h))

    # Write the frames to the video
    for i in range(video_frames.shape[1]):
        frame = video_frames[0][i]
        video_writer.write(frame)

    # Close the video writer
    video_writer.release()
video_path = export_to_video(video_frames, output_video_path="output.mp4", fps=24)